In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport data_processing
%aimport codelists
%aimport constants
%aimport import_file

In [3]:
from data_processing import *

In [4]:
from import_file import *

In [5]:
pd.options.display.max_rows = 1000

In [6]:
# clean_matching()

In [7]:
# create_pegmed()

In [8]:
# create_pegprod()

In [9]:
# create_medcoded_entries()

In [10]:
# create_prescriptions()

In [11]:
# create_consultations()

In [12]:
# create_all_entries()

In [13]:
all_entries = pd.read_csv('data/pt_data/all_entries.csv',delimiter=',')

In [14]:
# all_entries.sample(5)

In [15]:
# create_pt_features()

In [16]:
pt_features = pd.read_csv('data/pt_data/pt_features.csv',delimiter=',')

In [17]:
pt_features.sample(5)

,patid,pracid,gender,birthyear,index_date,isCase
33970,1441140,140,1,1928,2006-08-21,True
16960,8497487,487,2,1919,2002-04-02,True
58202,2170538,538,1,1938,NaN,False
32834,12101315,315,2,1931,2008-05-27,True
60734,4014219,219,1,1920,NaN,False


In [18]:
pegmed = pd.read_csv('data/dicts/proc_pegasus_medical.csv')

In [19]:
pegprod = pd.read_csv('data/dicts/proc_pegasus_prod.csv',delimiter=',')

In [20]:
medcodes = get_medcodes_from_readcodes(codelists.dementia_readcodes)

In [ ]:
prodcodes = get_prodcodes_from_drug_name(codelists.antidementia_drugs)

In [ ]:
all_dementia_entries = all_entries[(all_entries['prodcode'].isin(prodcodes))|(all_entries['medcode'].isin(medcodes))]

In [ ]:
all_dementia_entries.sample(5)

In [ ]:
all_dem_labelled = pd.merge(all_dementia_entries,pegmed,how='left')[['patid','prodcode','medcode','sysdate','eventdate','type','read term']]

In [ ]:
all_dem_labelled = pd.merge(all_dem_labelled,pegprod,how='left')[['patid','medcode','prodcode','sysdate','eventdate','type','drug substance name','read term']]

In [ ]:
def get_earliest_dementia_entry(patid):
    mask = all_dem_labelled['patid']==patid
    patient_entries = all_dem_labelled[mask]
    if len(patient_entries)>0:
        patient_entries.sort_values(by='eventdate',ascending=False, inplace=True)
        earliest_entry = patient_entries.head(1)
        return earliest_entry

In [ ]:
def get_final_dementia_dx(patid):
    mask = (all_dem_labelled['patid']==patid) & (all_dem_labelled['type']!=constants.entry_type['prescription'])
    patient_diagnoses = all_dem_labelled[mask]
    if len(patient_diagnoses)>0:
        patient_diagnoses.sort_values(by='eventdate',ascending=True, inplace=True)
        latest_dx = patient_diagnoses.head(1)
        return latest_dx

In [ ]:
# check_pt_has_dementia_entries = lambda x: True if len(all_dem_labelled[all_dem_labelled['patid']==x])>0 else False

In [ ]:
# pt_features['verified_isCase'] = pt_features['patid'].map(check_pt_has_dementia_entries )

In [ ]:
# pt_features

In [ ]:
cases_mask = pt_features['verified_isCase']==True

In [ ]:
pt_features.loc[cases_mask,'verified_index_date'] = pt_features.loc[cases_mask,'patid'].map(get_earliest_dementia_entry)

In [ ]:
pt_features.loc[cases_mask,'final_dementia_dx'] = pt_features.loc[cases_mask,'patid'].map(get_final_dementia_dx)

In [ ]:
pt_features

In [ ]:
# add_length_of_data_pre_and_post_indexdate_to_pt_features(isCase=True)

In [ ]:
# delete_patients_if_not_enough_data(isCase=True)

In [ ]:
# match_cases_and_controls()

In [ ]:
# delete_unmatched_controls()

In [ ]:
# add_length_of_data_pre_and_post_indexdate_to_pt_features(isCase=False)

In [ ]:
# delete_patients_if_not_enough_data(isCase=False)

In [ ]:
# create_insomnia_entries()

In [ ]:
# add_insomnia_event_count_to_pt_features()